In [1]:
import import_ipynb #Allows you to import ipynb files (TEMPORARY?)
import numpy as np
from queue import PriorityQueue
from Node import Node

importing Jupyter notebook from Node.ipynb


In [2]:
#This will implement the basic A* Algorithm
class HybridAStar:
    #class variables
    def __init__(self, grid, start, end):
        self.grid = grid
        self.start = start
        self.end = end
        
    #gets all neighbors at current node
    def GetNeighborsAtPosition(self, current_node):
        neighbors = []
        for new_position in [(0, -1), (0, 1), (-1, 0), (1, 0), (-1, -1), (-1, 1), (1, -1), (1, 1)]:
            node_position = (current_node.position[0] + new_position[0], current_node.position[1] + new_position[1])
            #out of bounds check
            if node_position[0] > (len(self.grid) - 1) or node_position[0] < 0 or node_position[1] > (len(self.grid[len(self.grid) - 1]) - 1) or node_position[1] < 0:
                continue
            #obstacle check
            if self.grid[node_position[0]][node_position[1]] != 0:
                continue
            new_node = Node(current_node, node_position)
            neighbors.append(new_node)
        return neighbors
    
    #Generates the path
    def GeneratePath(self):
        start_node = Node(None, self.start)
        end_node = Node(None, self.end)
        open_list = PriorityQueue()
        closed_list = []
        open_list.put(start_node)
        
        while open_list.qsize() > 0:
            current_node = open_list.get()
            closed_list.append(current_node)
            
            if(current_node == end_node):
                path = []
                current = current_node
                while current is not None:
                    path.append(current.position)
                    current = current.parent
                return path[::-1]
            
            #Get all open paths O(8*n) there will always be at most 8 neighbors, and at most will check n-size, its 8*n, O(n)
            neighbors = self.GetNeighborsAtPosition(current_node)
            open_neighbors = neighbors
            for index, neighbor in enumerate(neighbors):
                for closed in closed_list:
                    if neighbor == closed:
                        open_neighbors.pop(index)
                        
            #calculate cost of each path and add to queue
            for neighbor in open_neighbors:
                neighbor.g = current_node.g + 1
                neighbor.h = ((neighbor.position[0] - end_node.position[0]) ** 2) + ((neighbor.position[1] - end_node.position[1]) ** 2)
                neighbor.f = neighbor.g + neighbor.h
                
                open_list.put(neighbor)         

In [3]:
#Overall Algorithm
#First the car will report any obstacles in its way
#if any, add to map w/ appropriate location
#then perform a* algorithm with global map
#do a check if there's a collision
#if so, run machine learning algorithm to change, unless there are no changes to the map
#Should only use machine learning algo IF and only IF there are changes to the map
#return path to car, use route instructions to move car appropriately

In [4]:
#To-Do:
#figure out how to optimize this for cars (take steering to account)
#figure out how to make initial node fat
#if there is a better algorithm, since A* is memory intensive, do it